In [2]:
import numpy as np
from skimage import measure
from stl import mesh as stl_mesh
import plotly.graph_objects as go  # for visualization
import os
import open3d as o3d
import trimesh


import gyroid_utils
from gyroid_utils.utils import reload_all
reload_all()

working_path = os.getcwd()
print("Current working directory:", working_path)




[gyroid_utils] version 0.1.0 loaded
gyroid_utils: all modules reloaded
Current working directory: c:\Users\cofo\Documents\02 - GitHub\GYROIDS


#==============================
#======= design space =========
#==============================

In [8]:
# size of domain
pz2 = 200
py2 = 100
px2 = 100

# --- Discretization of the domain ---
# Resolution in each axis, calculated as a funcion of the size of the gyroid's unit cell
dx_grid = px2 / 200
dy_grid = py2 / 200
dz_grid = pz2 / 400

# 1D coordinate arrays. np.arange(stop + step, step) includes the endpoint like MATLAB's colon with step.
x1 = np.arange(0, px2 + dx_grid, dx_grid)       # x positions from 0 to pz2 + dx_grid, and the step size is dx_grid
y1 = np.arange(0, py2 + dy_grid, dy_grid)       # y positions from 0 to py2, step dy_grid
z1 = np.arange(0, pz2 + dz_grid, dz_grid)       # z positions from 0 to pz2, step dz_grid

# Create 3D coordinate grids. indexing='ij' -> (X,Y,Z) follow x1,y1,z1 order like MATLAB.
x, y, z = np.meshgrid(x1, y1, z1, indexing='ij')
print(np.min(x),np.max(x))
print(np.min(y),np.max(y))
print(np.min(z),np.max(z))

print(f"x-axis resolution {np.size(x1)=}, y-axis resolution {np.size(y1)=}, z-axis resolution {np.size(z1)=}")
print(f"In total, {np.size(x)} voxels in the 3D grid")

0.0 100.0
0.0 100.0
0.0 200.0
x-axis resolution np.size(x1)=201, y-axis resolution np.size(y1)=201, z-axis resolution np.size(z1)=401
In total, 16200801 voxels in the 3D grid


#===============================
#======= define gyroid =========
#===============================

In [ ]:
# --------- Create gyroids -----------
file_name = "gyroid-diagnal-1"

# ---  Define Period of gyroid unit cell -------
px = np.zeros_like(x) + 15
py = np.zeros_like(y) + 15
pz = np.zeros_like(z) + 15


# ------- check for errors -------
#in the period of the gyroid is too small for the grid resolution, it will cause errors in the marching cubes algorithm
if np.min(pz) <= 5 * dz_grid :
    print(f"max pz: {np.max(pz)}, min pz: {np.min(pz)}, resultion dz_grid: {dz_grid}")
    raise ValueError("Period too small for the grid resolution. z-axis.")   
elif np.min(py) <= 5 * dy_grid :
    print(f"max pz: {np.max(py)}, min pz: {np.min(py)}, resultion dz_grid: {dy_grid}")
    raise ValueError("Period too small for the grid resolution. y-axis.")   
elif np.min(px) <= 5 * dx_grid:
    print(f"max pz: {np.max(px)}, min pz: {np.min(px)}, resultion dz_grid: {dx_grid}")
    raise ValueError("Period too small for the grid resolution. z-axis.")

# ------ Define Thickness function t ---   #can be from -1.4265847744427516 + 1.4265847744427516

angle = 1      # smaller = flatter diagonal
shift = 50       # shifts it off the origin
sigma = 20          # controls smoothness (bigger = smoother)
amp   = 1.5
center = shift/angle

d = angle*x - z + shift
t = amp * np.exp(-(d**2) / (2 * sigma**2))

t = (t + 0.4*np.max(t)) / (np.max(t)*0.9)
# top and bottom base
t[:,:,0:10] = np.max(t)
t[:,:,-11:] = np.max(t)


print(np.min(t))
print(np.max(t))
t[t < 0] = 0
t[t > 1.5] = 1.5


# --- Gyroid scalar field v (isosurface at v=0 gives the gyroid surface) ---
v = np.abs( np.sin((2*np.pi/px)*x) * np.cos((2*np.pi/py)*y)
    + np.sin((2*np.pi/py)*y) * np.cos((2*np.pi/pz)*z)
    + np.sin((2*np.pi/pz)*z) * np.cos((2*np.pi/px)*x) ) - t # add thickness field
v=-v

# --- Choose the isovalue (same as MATLAB's "isosurface(..., 0)") ---
iso_level = 0

# --- Visualize result ---

gyroid_utils.viz.twod_view_of_matrix(v, x, y, z, 0, 0.01)

#===============================================
#======= Make stl-style mesh (surface) =========
#===============================================

In [12]:
# --- Ensure the surface is CLOSED (emulates MATLAB's isocaps(..., "below")) ---
# Strategy: we want the interior to be "v < iso_level" (the "below" side).
# If the surface hits the boundary, it's open. We close it by padding a 1-voxel thick
# frame of large POSITIVE values (>> iso_level) around v.
# That guarantees a crossing at the boundary and thus creates "caps" there.
pad_val = 0                                              # safely above any v near 0
v_padded = np.pad(v, pad_width=5, mode='constant', constant_values=pad_val)

# Because we padded the volume, the new grid extends one extra voxel outward on each face.
# The physical spacing of voxels is the same as our grid steps:
spacing = (dx_grid, dy_grid, dz_grid)                        # real-world size per voxel

# The physical origin of the padded grid is shifted by -1 voxel in each direction:
origin = (x1.min() - dx_grid, y1.min() - dy_grid, z1.min() - dz_grid)


# --- Extract the isosurface in one pass, scaled to physical units ---
# Using 'spacing' returns vertices already scaled; we still need to add 'origin'.
verts, faces, normals, values = measure.marching_cubes(
    v_padded, level=iso_level, spacing=spacing, step_size=3, allow_degenerate=False)

# Translate vertices so they live in the same physical coordinate system as (x,y,z)
verts[:, 0] += origin[0]
verts[:, 1] += origin[1]
verts[:, 2] += origin[2]

print(f"there are {len(faces)} faces in this model")

# simplify and clean the mesh
faces, verts = gyroid_utils.mesh_tools.simplify_mesh(faces, verts, target=300000)
verts, faces = gyroid_utils.mesh_tools.keep_largest_connected_component(verts, faces)

# ---- save the mesh ----
gyroid_utils.viz.save_mesh_as_html(faces, verts, file_name)
gyroid_utils.mesh_tools.export_as_STL(verts, faces, f"{working_path}/{file_name}.stl")

# ------ visualize the quality of your mesh ----
triangle_areas = gyroid_utils.mesh_tools.calculate_triangle_areas(verts, faces)
gyroid_utils.viz.plot_histogram(triangle_areas)


there are 961552 faces in this model
[INFO] gyroid_utils: Simplifying mesh: 961552 faces → target 300000
[INFO] gyroid_utils: Mesh simplification complete → 300000 faces remain.
[INFO] gyroid_utils: Extracting largest connected component…
[INFO] gyroid_utils: Selected largest component: 131940 vertices, 300000 faces
[INFO] gyroid_utils: Saving mesh visualization → 'gyroid-diagnal-1.html'
[INFO] gyroid_utils: Reducing edges: 450000 → 100000 (distance-based sampling)
[INFO] gyroid_utils: HTML visualization saved → gyroid-diagnal-1.html
[INFO] gyroid_utils: Exporting STL → c:\Users\cofo\Documents\02 - GitHub\GYROIDS/gyroid-diagnal-1.stl
[INFO] gyroid_utils: STL successfully saved: c:\Users\cofo\Documents\02 - GitHub\GYROIDS/gyroid-diagnal-1.stl
[INFO] gyroid_utils: Calculating triangle areas…
[INFO] gyroid_utils: Triangle area computation complete.
[INFO] gyroid_utils: Plotting histogram for 300000 triangle areas


[INFO] gyroid_utils: Histogram displayed successfully.


#=========================================
#======= visualize your creation =========
#=========================================

In [ ]:
fig_2 = go.Figure()
fig_2.add_trace(go.Scatter(
    x=x[:,0,0],
    y=v[:,0,0],
    mode='lines+markers',
    name='Example Line'
))
fig_2.update_layout(title="v function along x-axis",
                    template="plotly_white")
fig_2.show()

In [ ]:
fig_0 = go.Figure()
fig_0.add_trace(go.Scatter(
    x=x[:,0,0],
    y=t[:,0,0],
    mode='lines+markers',
    name='x axis'
))
fig_0.add_trace(go.Scatter(
    x=y[0,:,0],
    y=t[0,:,0],
    mode='lines+markers',
    name='y axis'
))
fig_0.add_trace(go.Scatter(
    x=z[0,0,:],
    y=t[0,0,:],
    mode='lines+markers',
    name='z axis'
))
fig_0.update_layout(title="Thickness function along x,y and z-axis",
                    template="plotly_white")
fig_0.show()

